In [34]:
import ecephys_analyses as ea

In [35]:
subject = "Allan"
experiment = "ketamine"
condition = "all"

In [94]:
from ecephys.sglx_utils.external.readSGLX import readMeta

def write_empty_hypnogram(bin_path, hypnogram_path, default_state="None"):
    meta = readMeta(bin_path)
    duration = round(float(meta["fileTimeSecs"]), 2)
    with open(hypnogram_path, 'x') as f:
        f.write(f"*Duration_sec\t{duration}")
        f.write("\n")
        f.write(f"{default_state}\t{duration}")
        f.write("\n")

In [36]:
bin_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, ext="lf.bin")
hypnogram_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, ext="hypnogram.txt")

In [ ]:
for bin_path, hypnogram_path in zip(bin_paths, hypnogram_paths):
    write_empty_hypnogram(bin_path, hypnogram_path)

In [90]:
from pathlib import PureWindowsPath

def translate_server_path(path, drive):
    data_dir_idx = path.parts.index('Data')
    path_parts = path.parts[data_dir_idx:]
    windows_path = PureWindowsPath(drive).joinpath(*path_parts)
    return str(windows_path)

In [106]:
import yaml

def write_sleepscore_config(cfg_path, bin_path, emg_path, bin_drive='U:\\', emg_drive='W:\\', chan_list=[], states_config_file='W:\Data\sleepscore_states_with_drug.yml'):
    dataset = dict(binPath=translate_server_path(bin_path, bin_drive), datatype="SGLX", chanList=chan_list)
    config = dict(
        datasets=[dataset],
        downSample=100.0,
        ds_method="interpolation",
        tStart=0.0,
        tEnd=None,
        EMGdatapath=translate_server_path(emg_path, emg_drive),
        kwargs_sleep=dict(states_config_file=states_config_file),
    )
    with open(cfg_path, "x") as f: 
        yaml.dump(config, f, sort_keys=False)

In [99]:
bin_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, ext="lf.bin")
emg_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, ext="emg.nc")
cfg_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, ext="sleepscore-config.yml")

In [ ]:
for bin_path, emg_path, cfg_path in zip(bin_paths, emg_paths, cfg_paths):
    write_sleepscore_config(cfg_path, bin_path, emg_path)